In [1]:
#!pip install open3d
#!conda install -c open3d-admin open3d

In [1]:
import open3d as o3d
import numpy as np
import copy

In [2]:
#pointCloudMatrix = [[0.0, 0, 0],[0, 0, 1],[0, 1, 0],[0, 1, 1],[1, 0, 0],[1, 0, 1],[1, 1, 0],[1, 1, 1]];

#pointCloudMatrix = np.array([[0,0,0]])
#for x in range(51):
   # for y in range(51):
       # for z in range(51):
          #  if (abs(x/50) == 1 or abs(y/50) == 1 or abs(z/50)==1 or abs(x) == 0 or abs(y) == 0 or abs(z)==0):
              #  pointCloudMatrix = np.concatenate((pointCloudMatrix,[[x/3,y/3,z/3]]),axis=0)
#pointCloudMatrix = np.array(np.loadtxt('np_array.txt'))                
pointCloudMatrix = np.array(np.loadtxt('vertices-hdroomfixed-64mation.txt'))            


In [3]:
pointCloudMatrix.shape

(18991, 6)

In [4]:
color = pointCloudMatrix[:,3:6]
pointCloudMatrix= pointCloudMatrix[:,0:3]

pointCloudMatrix.shape



(18991, 3)

In [5]:
def simplistic_mesh_creator_voxel(pointCloudMatrix,suppressOutputs=True):
     #takes a numpy matrix of size [x,3] of points 
     #and uses open3d inorder to create a basic voxel representation of the points
    
    o3dPointCloud = o3d.geometry.PointCloud() #Create pointcloud object
    o3dPointCloud.points = o3d.utility.Vector3dVector(pointCloudMatrix) #Populate with points from numpy matrix
    o3dPointCloud.scale(1 / np.max(o3dPointCloud.get_max_bound() - o3dPointCloud.get_min_bound()),
          center=o3dPointCloud.get_center()) #fix the scale
    #o3dPointCloud.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
    #voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,
                                                            #voxel_size=0.05)
    
    v_size = round(max(o3dPointCloud.get_max_bound()-o3dPointCloud.get_min_bound())*0.005,4) #find Side of each voxel
    
    voxel_grid=o3d.geometry.VoxelGrid.create_from_point_cloud(o3dPointCloud,voxel_size=v_size)
    # o3d.visualization.draw_geometries([voxel_grid]) #visualization
    
    voxels=voxel_grid.get_voxels()
    vox_mesh = o3d.geometry.TriangleMesh()
    #Generate Meshes from the voxels
    for v in voxels:
        cube = o3d.geometry.TriangleMesh.create_box(width=1, height=1,depth=1)
        cube.paint_uniform_color(v.color)
        cube.translate(v.grid_index, relative=False)
        vox_mesh+=cube
    
    vox_mesh.translate([0.5,0.5,0.5], relative=True)
    vox_mesh.scale(v_size, [0,0,0])
    vox_mesh.translate(voxel_grid.origin, relative=True)
    vox_mesh.merge_close_vertices(0.0000001) 
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([vox_mesh]) #visualization
    o3d.io.write_triangle_mesh("Voxel.obj", vox_mesh)
    return vox_mesh;

    

In [6]:
Out_Vox = simplistic_mesh_creator_voxel(pointCloudMatrix,False)

In [7]:
import matplotlib.pyplot as plt

In [8]:
def simplistic_mesh_creator_poisson(pointCloudMatrix,suppressOutputs=True):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())
    #o3d.visualization.draw_geometries([pcd])

    pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3))) #remove exiting Normals
    #pcd.estimate_normals() #Estimate new normals base on pointcloud
    radius = 0.1   # 
    max_nn = 30   # 
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))
    pcd.orient_normals_consistent_tangent_plane(100) #Takes a while but works!
    
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
            mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
                pcd, depth=9)
    if(not suppressOutputs):        
        o3d.visualization.draw_geometries([mesh])
        print('visualize densities')
    densities = np.asarray(densities)
    density_colors = plt.get_cmap('plasma')(
    (densities - densities.min()) / (densities.max() - densities.min()))
    density_colors = density_colors[:, :3]
    density_mesh = o3d.geometry.TriangleMesh()
    
    density_mesh.vertices = mesh.vertices
    density_mesh.triangles = mesh.triangles
    density_mesh.triangle_normals = mesh.triangle_normals
    density_mesh.vertex_colors = o3d.utility.Vector3dVector(density_colors)
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    
    rough_Density_Mesh = copy.deepcopy(density_mesh);
    
    o3d.io.write_triangle_mesh("density_mesh_possian_full.ply", density_mesh)
    
    
    print('remove low density vertices')

    #o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    vertices_to_remove = densities < np.quantile(densities, 0.05) #best Density For most cases
    density_mesh.remove_vertices_by_mask(vertices_to_remove)
    print(mesh)
    if(not suppressOutputs):
        o3d.visualization.draw_geometries([density_mesh],mesh_show_back_face=True)
    o3d.io.write_triangle_mesh("density_mesh_possian_reduced.ply", density_mesh)
    return pcd, mesh, rough_Density_Mesh, density_mesh;
   

In [9]:
pointCloud_EstimatedNormals,Out_PosRough,Out_PosRColored,Out_Pos=simplistic_mesh_creator_poisson(pointCloudMatrix)

[Open3D DEBUG] Input Points / Samples: 18991 / 18899
[Open3D DEBUG] #   Got kernel density: 0.105000 (s), 185.008 (MB) / 185.008 (MB) / 318 (MB)
[Open3D DEBUG] #     Got normal field: 0.039 (s), 192.082 (MB) / 192.082 (MB) / 318 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 2.082098e-04 / 3.954113e+00
[Open3D DEBUG] #       Finalized tree: 0.12 (s), 199.34 (MB) / 199.34 (MB) / 318 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.181 (s), 193.844 (MB) / 199.34 (MB) / 318 (MB)
[Open3D DEBUG] #Set point constraints: 0.033 (s), 191.426 (MB) / 199.34 (MB) / 318 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 546603 / 239416 / 385273
[Open3D DEBUG] Memory Usage: 191.426 MB
[Open3D DEBUG] # Linear system solved: 0.356 (s), 205.031 (MB) / 205.031 (MB) / 318 (MB)
[Open3D DEBUG] Got average: 0.0119998 (s), 189.766 (MB) / 205.031 (MB) / 318 (MB)
[Open3D DEBUG] Iso-Value: 5.106812e-01 = 9.698345e+03 / 1.899100e+04
[Open3D DEBUG] #          Total Solve:       1.6 (s),     232.7 (MB)


In [78]:
def generateBallPivot(pointCloud_With_Estimated_Normals):
    radii = [0.005, 0.01, 0.02, 0.04,0.08]
    rec_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
        pointCloud_EstimatedNormals, o3d.utility.DoubleVector(radii))
    o3d.visualization.draw_geometries([pointCloud_With_Estimated_Normals, rec_mesh])
    return rec_mesh

In [79]:
Out_BallPivot = generateBallPivot(pointCloud_EstimatedNormals);

In [44]:
def generateConvexHull(pointCloudMatrix):

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointCloudMatrix)
    pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()), center=pcd.get_center())

    hull, _ = pcd.compute_convex_hull()
    hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
    hull_ls.paint_uniform_color((0, 0, 1))
    o3d.visualization.draw_geometries([hull,hull_ls],mesh_show_back_face=True)
    o3d.io.write_triangle_mesh("ConvexHull.ply", hull)
    return hull;

In [48]:
Out_Hull = generateConvexHull(pointCloudMatrix)

In [12]:
import os
import sys
import torch
!pip install pytorch3d
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes,cubify
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
    point_mesh_face_distance
)
import numpy as np
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from tqdm.notebook import tqdm

In [13]:
# Set the device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be slow!")

In [14]:

src_obj = os.path.join('voxel.obj')
verts, faces, aux = load_obj(src_obj)
faces_idx = faces.verts_idx.to(device)
verts = verts.to(device)
src_mesh = Meshes(verts=[verts], faces=[faces_idx])


trg_mesh = torch.tensor([pointCloudMatrix.tolist()])
target_point_count = trg_mesh.size(dim=1)

sample_trg = trg_mesh
sample_src = sample_points_from_meshes(src_mesh, 5000)


loss_chamfer, _ = chamfer_distance(sample_trg, sample_src)

print(loss_chamfer);

tensor(57.7663)


In [89]:

Out_Hull_Translated = Out_Hull
Out_Hull_Translated.translate([1.5,0,0])
Out_BallPivot.translate([1.5*2,0,0])
Out_PosRough.translate([1.5*3,0,0])
Out_PosRColored.translate([1.5*4,0,0])
Out_Pos.translate([1.5*5,0,0])


o3d.visualization.draw_geometries([Out_Vox, Out_Hull_Translated,Out_BallPivot,Out_PosRough,Out_PosRColored,Out_Pos],mesh_show_back_face=True)
Out_Hull_Translated.translate([-1.5,0,0])
Out_BallPivot.translate([1.5*-2,0,0])
Out_PosRough.translate([-1.5*3,0,0])
Out_PosRColored.translate([-1.5*4,0,0])
Out_Pos.translate([-1.5*5,0,0])



TriangleMesh with 50469 points and 99715 triangles.

In [ ]:
#indiviudaul Display For Demo: Vox
o3d.visualization.draw_geometries([Out_Vox],mesh_show_back_face=True)

In [72]:
#indiviudaul Display For Demo: Hull
o3d.visualization.draw_geometries([Out_Hull],mesh_show_back_face=True)

In [86]:
#indiviudaul Display For Demo: Estimated Normals:
o3d.visualization.draw_geometries([pointCloud_EstimatedNormals], point_show_normal=True)

In [83]:
#indiviudaul Display For Demo: BallPivot
o3d.visualization.draw_geometries([Out_BallPivot],mesh_show_back_face=True)

In [87]:
#indiviudaul Display For Demo: Poisson
o3d.visualization.draw_geometries([Out_PosRColored],mesh_show_back_face=True)

In [10]:
#indiviudaul Display For Demo: Poisson Removed Low Densities
o3d.visualization.draw_geometries([Out_Pos],mesh_show_back_face=True)